### קוד מבוא

#### ספריות

In [1]:
import os
import sys
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

#### העלאת משתנים להרצת הקוד

In [3]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

create_forecast_basic_software_location = os.path.dirname(future_folder)

sys.path.append(create_forecast_basic_software_location)

create_forecast_basic_software_df = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(create_forecast_basic_software_location))

TAZ_V4_date = create_forecast_basic_software_df['location'][1]

### פונקציות

### פונקציות גלובליות

In [4]:
from global_functions import up_load_shp, up_load_df, drop_geo

### ביצוע

#### עיבוד מידע טבלאי של הדמוגרף

In [5]:
demo=up_load_df(r'{}\background_files'.format(software_data_folder_location),'‏‏PalestiniansResults_forecast_zone')

In [37]:
col=['male_2020',
 'female_2020',
 'male_2025',
 'female_2025',
 'male_2030',
 'female_2030',
 'male_2035',
 'female_2035',
 'male_2040',
 'female_2040',
 'male_2045',
 'female_2045',
 'male_2050',
 'female_2050']

demo=demo.pivot_table(index='Proj_Area',aggfunc=sum)[col]

C:\Users\dpere\AppData\Local\Temp\ipykernel_20632\858144232.py:16: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  demo=demo.pivot_table(index='Proj_Area',aggfunc=sum)[col]


In [38]:
year=['2020','2025','2030','2035','2040','2045','2050']

for x in year:
    demo['pop_{}'.format(x)]= demo['female_{}'.format(x)]+demo['male_{}'.format(x)]

#### שכבת אזורי תחזית של דמוגרף

In [39]:
proj_zones=up_load_shp(r'{}\background_files\proj_zones_pls.shp'.format(software_data_folder_location))

proj_zones['Proj_Area']=proj_zones['projection']

proj_zones=proj_zones[['Proj_Area', 'geometry']]

#### חיבור נתוני דמוגרף

In [40]:
pd.merge(proj_zones,demo,on='Proj_Area',how='right').query('geometry.isna()')

,Proj_Area,geometry,male_2020,female_2020,male_2025,female_2025,male_2030,female_2030,male_2035,female_2035,male_2040,female_2040,male_2045,female_2045,male_2050,female_2050,pop_2020,pop_2025,pop_2030,pop_2035,pop_2040,pop_2045,pop_2050
36,50,None,"1,232.43","1,124.17","1,379.29","1,269.85","1,546.52","1,436.05","1,738.47","1,625.90","1,942.32","1,827.51","2,142.52","2,026.16","2,334.97","2,216.95","2,356.60","2,649.13","2,982.57","3,364.37","3,769.84","4,168.67","4,551.91"


 אנחנו מוכנים לוותר עליהם 
 זה ניקוז של כל הבדואיים שאין להם אזור תחזית

In [41]:
proj_zones=pd.merge(proj_zones,demo,on='Proj_Area',how='left')

#### שכבת אזורי תנועה

In [42]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date))

taz=taz.query('main_secto=="Palestinian"')[['Taz_num','geometry']]

#### שכבת בינוי

In [43]:
bld=up_load_shp(r'{}\background_files\palestinian_bld_area_pcbs_220710.shp'.format(software_data_folder_location))

bld=bld.to_crs(2039)

bld=bld.dissolve()

bld=bld[['geometry']]

#### הצלבת בין בינוי לבין אזורי תחזית ותנועה

In [44]:
col=['Proj_Area',
 'geometry',
 'pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']

demo_taz_bld=bld.overlay(taz).overlay(proj_zones[col])

demo_taz_bld['small_area']=demo_taz_bld.area

demo_taz_bld=demo_taz_bld.set_index('Proj_Area')

demo_taz_bld['proj_sum_area']=drop_geo(demo_taz_bld).groupby(by='Proj_Area').sum()[['small_area']]

demo_taz_bld['pre_from_proj']=demo_taz_bld['small_area']/demo_taz_bld['proj_sum_area']

c:\Users\dpere\miniconda3\envs\creat_forecast_ad_hoc_env\Lib\site-packages\geopandas\geodataframe.py:2469: UserWarning: `keep_geom_type=True` in overlay resulted in 4 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [45]:
pop_year=['pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']

for y in pop_year:
    demo_taz_bld['{}'.format(y)]=demo_taz_bld['{}'.format(y)]*demo_taz_bld['pre_from_proj']

בדיקה שלא הומצא כמות אנשים

In [46]:
demo_taz_bld[pop_year].sum().sum()-proj_zones[pop_year].sum().sum()

3.725290298461914e-09

#### סכום אנשים לפי אזורי תנועה

In [47]:
taz_demo_pls=drop_geo(demo_taz_bld).pivot_table(index='Taz_num',aggfunc=sum)

C:\Users\dpere\AppData\Local\Temp\ipykernel_20632\1244303127.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz_demo_pls=drop_geo(demo_taz_bld).pivot_table(index='Taz_num',aggfunc=sum)


In [48]:
add_year=['2025', '2030', '2035', '2040', '2045', '2050']

x=2020

for y in add_year:
    taz_demo_pls['precent_add_pop_{}'.format(y)]=taz_demo_pls['pop_{}'.format(y)]/taz_demo_pls['pop_{}'.format(str(x))]
    taz_demo_pls['precent_add_pop_{}'.format(y)]=taz_demo_pls['precent_add_pop_{}'.format(y)].round(2)
    taz_demo_pls['precent_add_pop_{}'.format(y)].fillna(0)
    x+=5

#### ייצוא

In [49]:
col=['pop_2020',
 'precent_add_pop_2025',
 'precent_add_pop_2030',
 'precent_add_pop_2035',
 'precent_add_pop_2040',
 'precent_add_pop_2045',
 'precent_add_pop_2050']

In [50]:
r'{}\Intermediates\taz_demo_pls_2020_and_pre_growth_till_2050.xlsx'.format(software_data_folder_location)

'C:\\Users\\dpere\\Documents\\JTMT\\forecast_git\\create_forecast_basic\\future\\arab_and_palestinian\\Intermediates\\taz_demo_pls_2020_and_pre_growth_till_2050.xlsx'

In [51]:
taz_demo_pls[col].to_excel(r'{}\Intermediates\taz_demo_pls_2020_and_pre_growth_till_2050.xlsx'.format(software_data_folder_location))